In [2]:
import os
import ultralytics
from ultralytics import YOLO
import zipfile

In [3]:
import torch

# cuDNN 비활성화
torch.backends.cudnn.enabled = False

In [ ]:
import os
import torch
from PIL import Image
from torchvision import models, transforms
from torchvision.utils import save_image
from tqdm import tqdm


# Mask R-CNN 모델 로드
model = models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

# 이미지 전처리 함수 정의
transform = transforms.ToTensor()

# 데이터셋 경로 설정
dataset_dirs = ['train', 'valid', 'test']
base_path = 'C:/Users/SBA/repository/image_detection/vehicle dataset'

# 각 데이터셋 디렉토리를 순회하며 이미지 파일 처리
for dataset in dataset_dirs:
    image_dir = os.path.join(base_path, dataset, 'images')
    mask_dir = os.path.join(base_path, dataset, 'masks')

    # 마스크 폴더가 없으면 생성
    if not os.path.exists(mask_dir):
        os.makedirs(mask_dir)

    # 이미지 파일을 하나씩 처리
    for image_file in os.listdir(image_dir):
        if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(image_dir, image_file)
            image = Image.open(image_path).convert('RGB')
            
    # # tqdm을 사용하여 이미지 파일 처리 진행 상황 표시
    # image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    # for image_file in tqdm(image_files, desc=f"Processing {dataset} images"):
    #     image_path = os.path.join(image_dir, image_file)
    #     image = Image.open(image_path).convert('RGB')

            # 이미지 텐서로 변환
            image_tensor = transform(image).unsqueeze(0)

            # 모델을 사용하여 마스크 예측
            with torch.no_grad():
                prediction = model(image_tensor)

            # 첫 번째 객체의 마스크 저장
            if len(prediction[0]['masks']) > 0:  # 마스크가 존재하는 경우만 저장
                mask = prediction[0]['masks'][0, 0]
                mask_filename = os.path.splitext(image_file)[0] + '.png'  # 이미지 파일명과 동일하게
                mask_path = os.path.join(mask_dir, mask_filename)
                save_image(mask, mask_path)
                print(f"Saved mask for {image_file} to {mask_path}")
            else:
                print(f"No objects detected in {image_file}, skipping mask creation.")

In [4]:
# YOLOv8 모델 불러오기 (자동으로 다운로드)
model = YOLO('yolov8n-seg.pt') 

# 모델 학습 수행
model.train(data='vehicle dataset\data.yaml', epochs=150, batch = 8, lr0 = 0.001)

New https://pypi.org/project/ultralytics/8.3.7 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.6  Python-3.10.0 torch-2.4.1+cpu CPU (12th Gen Intel Core(TM) i7-12700)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=vehicle dataset\data.yaml, epochs=150, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sav

train: Scanning C:\Users\SBA\repository\image_detection\vehicle dataset\train\labels.cache... 878 images, 0 backgrounds, 0 corrupt: 100%|██████████| 878/878 [00:00<?, ?it/s]
val: Scanning C:\Users\SBA\repository\image_detection\vehicle dataset\valid\labels.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


Plotting labels to runs\segment\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 72 weight(decay=0.0), 83 weight(decay=0.0005), 82 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\segment\train4
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/110 [00:00<?, ?it/s]


TypeError: ERROR ❌ segment dataset incorrectly formatted or not a segment dataset.
This error can occur when incorrectly training a 'segment' model on a 'detect' dataset, i.e. 'yolo train model=yolov8n-seg.pt data=coco8.yaml'.
Verify your dataset is a correctly formatted 'segment' dataset using 'data=coco8-seg.yaml' as an example.
See https://docs.ultralytics.com/datasets/segment/ for help.

In [ ]:
# Load a model
# model = YOLO("yolo11n.pt")  # pretrained YOLO11n model
model = YOLO("runs/detect/train/weights/best.pt")

imagePath1 = 'person.jpg'
imagePath2 = 'bus.jpg'
# Run batched inference on a list of images
results = model([imagePath1, imagePath2])  # return a list of Results objects

# 2개의 이미지를 index값으로 각각 저장하기 위해 enumerate를 사용해서 인덱스값까지 얻어오기
# Process results list
for idx,result in enumerate(results):
    boxes = result.boxes  # 바운딩박스 정보 가져오기
    # masks = result.masks  # Masks object for segmentation masks outputs
    # keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # 분류확률 가져오기
    obb = result.obb  # obb정보 회전된 형태의 상자를 나타내며 객체의 방향을 고려한 경계상자
    result.show()  # 추론결과를 화면에 표시
    result.save(filename=f"result{idx+1}.jpg")  # 결과이미지를 파일로 저장